In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
# pd.set_option('display.max_rows', None)
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from transformers import AdamW
import torch.nn as nn
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

In [33]:
# df = pd.read_csv("/content/drive/MyDrive/University/4-2/정보기술학회/data/df_merge.csv", encoding = "utf-8")
df_1 = pd.read_csv("/content/drive/MyDrive/University/4-2/정보기술학회/data/df_real2.csv", encoding = "utf-8")
df_2 = pd.read_csv("/content/drive/MyDrive/University/4-2/정보기술학회/data/code1.csv", encoding = "utf-8")
A_code = pd.read_csv("/content/drive/MyDrive/University/4-2/정보기술학회/data/A.csv", encoding = "utf-8")

In [34]:
# 인덱스 붙인 파일
# df_1[df_1['주상병코드'] == 'A02'].head()

# 진료과목코드와 실제 진료과 매핑
mapping = {
    0: '일반의', 1: '내과', 2: '신경과', 3: '정신과', 4: '외과', 5: '정형외과', 6: '신경외과',
    7: '흉부외과', 8: '성형외과', 9: '마취통증의학과', 10: '산부인과', 11: '소아청소년과',
    12: '안과', 13: '이비인후과', 14: '피부과', 15: '비뇨기과', 16: '영상의학과',
    17: '방사선 종양학과', 18: '병리과', 19: '진단검사의학과', 20: '결핵과',
    21: '재활의학과', 22: '핵의학과', 23: '가정의학과', 24: '응급의학과',
    25: '산업의학과', 26: '예방의학과', 50: '구강악안면외과', 51: '치과보철과',
    52: '치과교정과', 53: '소아치과', 54: '치주과', 55: '치과보존과',
    56: '구강내과', 57: '구강악안면방사선과', 58: '구강병리과',
    59: '예방치과', 80: '한방내과', 81: '한방부인과',
    82: '한방소아과', 83: '한방안·이비인후·피부 과',
    84: '한방신경정신 과', 85: '침구 과',
    86: '한방재활의학 과',87:'사상체질 과',
    88:'한방응급 과','ZZ':'Nan'
}

# 진료코드를 실제 진료 과로 변환
df_1['진료과'] = df_1['진료과목코드'].map(mapping)

df_1.head()

,가입자일련번호,성별코드,연령대코드,진료과목코드,주상병코드,요양일수,입내원일수,총처방일수,한글명,진료과
0,416815,2,14,1,J20,1,1,4,급성 기관지염,내과
1,607165,1,14,1,J20,1,1,5,급성 기관지염,내과
2,301410,1,14,1,J20,1,1,3,급성 기관지염,내과
3,193953,2,14,1,J20,1,1,3,급성 기관지염,내과
4,859131,1,14,1,J20,1,1,2,급성 기관지염,내과


In [ ]:
# df_1.isna().sum()
# df_1.shape
# df_1 = df_1[df_1['주상병코드'].str.startswith(('J', 'A'))]

In [ ]:
# df_1['주상병코드'].value_counts()
# df.shape
# a_count = df_1[df_1['주상병코드'].str.startswith('J')].shape[0]
# print(f"Number of rows with 주상병코드 starting with 'A': {a_count}")

Number of rows with 주상병코드 starting with 'A': 3287809


In [ ]:
print(df_1.shape)
df_1.head()

(2105753, 10)


,가입자일련번호,성별코드,연령대코드,진료과목코드,주상병코드,요양일수,입내원일수,총처방일수,한글명,진료과
0,416815,2,14,1,J20,1,1,4,급성 기관지염,내과
1,607165,1,14,1,J20,1,1,5,급성 기관지염,내과
2,301410,1,14,1,J20,1,1,3,급성 기관지염,내과
3,193953,2,14,1,J20,1,1,3,급성 기관지염,내과
4,859131,1,14,1,J20,1,1,2,급성 기관지염,내과


In [7]:
#############
# 주상병코드 j #
#############

pd.set_option('display.max_columns', None)
df_2 = pd.read_csv("/content/drive/MyDrive/University/4-2/정보기술학회/data/code1.csv", encoding = "utf-8")
df_2.head()

,J00,"기침, 재채기, 콧물, 발열"
0,J01,"급성 부비동염의 증상은 상기도 감염 시에 나타나는 코막힘과 비루(콧물), 발열, 권..."
1,J02,"초기에는 인두의 이물감, 건조감, 가벼운 기침 정도의 증세가 나타나고, 심해지면 통..."
2,J03,"대부분의 경우 갑작스러운 고열과 오한이 나타난다. 뒤이어 인후통이 발생하고, 주변 ..."
3,J04,일반적으로 음식물이나 침을 삼킬 때 인두 이물감이나 통증이 발생한다. 급성 후두염을...
4,J05,"급성 후두개염의 대표적인 증상은 삼킬 때의 통증(odynophagia), 호흡곤란,..."


In [35]:
df_2 = df_2.rename(columns={'J00': '코드', '기침, 재채기, 콧물, 발열': '증상'})
new_row = pd.DataFrame([{'코드': 'J00', '증상': '코드, 기침, 재채기, 콧물, 발열'}])
df_2 = pd.concat([new_row, df_2]).reset_index(drop=True)
df_2.head()

,코드,증상
0,J00,"코드, 기침, 재채기, 콧물, 발열"
1,J01,"급성 부비동염의 증상은 상기도 감염 시에 나타나는 코막힘과 비루(콧물), 발열, 권..."
2,J02,"초기에는 인두의 이물감, 건조감, 가벼운 기침 정도의 증세가 나타나고, 심해지면 통..."
3,J03,"대부분의 경우 갑작스러운 고열과 오한이 나타난다. 뒤이어 인후통이 발생하고, 주변 ..."
4,J04,일반적으로 음식물이나 침을 삼킬 때 인두 이물감이나 통증이 발생한다. 급성 후두염을...


In [10]:
#############
# 주상병코드 a #
#############

A_code.head()

,코드,한글명,증상,Unnamed: 3
0,A02,기타 살모넬라감염,"감염자들은 메스꺼움 및 연축성 복통을 호소하며 그 후 장액성 설사, 열 및 구토 증...",NaN
1,A04,기타 세균성 장감염,"대장균 감염의 일반적 증상에는 구토, 구역, 복통, 설사, 두통 등이 있습니다. 균...",NaN
2,A05,달리 분류되지 않은 기타 세균성 음식매개중독,"다리에 출혈을 동반한 수포가 생기며, 고열, 패혈증 등이 나타납니다.심한 구토, 어...",NaN
3,A06,아메바증,"설사, 때때로 대변에서 보이는 혈액 동반경련이 있는 복통체중 감소 및 열",NaN
4,A07,기타 원충성 장질환,"증상을 보이는 질병은 복통, 복부 압통, 이급후증, 메스꺼움, 신경성 식욕부진, 피...",NaN


In [36]:
A_code = A_code.drop(columns=['한글명', 'Unnamed: 3'], axis=1)

A_code.head()

,코드,증상
0,A02,"감염자들은 메스꺼움 및 연축성 복통을 호소하며 그 후 장액성 설사, 열 및 구토 증..."
1,A04,"대장균 감염의 일반적 증상에는 구토, 구역, 복통, 설사, 두통 등이 있습니다. 균..."
2,A05,"다리에 출혈을 동반한 수포가 생기며, 고열, 패혈증 등이 나타납니다.심한 구토, 어..."
3,A06,"설사, 때때로 대변에서 보이는 혈액 동반경련이 있는 복통체중 감소 및 열"
4,A07,"증상을 보이는 질병은 복통, 복부 압통, 이급후증, 메스꺼움, 신경성 식욕부진, 피..."


In [37]:
# df_2와 A_code를 하나의 데이터프레임으로 병합
df_symptom = pd.concat([df_2, A_code], ignore_index=True)

# 주상병코드를 기준으로 증상 설명 매핑
df_1 = pd.merge(df_1, df_symptom[['코드', '증상']], left_on='주상병코드', right_on='코드', how='left')

# 불필요한 '코드' 열 제거
df_1 = df_1.drop(columns=['코드'])
df_1.head()

,가입자일련번호,성별코드,연령대코드,진료과목코드,주상병코드,요양일수,입내원일수,총처방일수,한글명,진료과,증상
0,416815,2,14,1,J20,1,1,4,급성 기관지염,내과,급성 기관지염에 걸리면 기관에 염증이 생기면서 점막이 붓고 점액이 분비된다. 이로 ...
1,607165,1,14,1,J20,1,1,5,급성 기관지염,내과,급성 기관지염에 걸리면 기관에 염증이 생기면서 점막이 붓고 점액이 분비된다. 이로 ...
2,301410,1,14,1,J20,1,1,3,급성 기관지염,내과,급성 기관지염에 걸리면 기관에 염증이 생기면서 점막이 붓고 점액이 분비된다. 이로 ...
3,193953,2,14,1,J20,1,1,3,급성 기관지염,내과,급성 기관지염에 걸리면 기관에 염증이 생기면서 점막이 붓고 점액이 분비된다. 이로 ...
4,859131,1,14,1,J20,1,1,2,급성 기관지염,내과,급성 기관지염에 걸리면 기관에 염증이 생기면서 점막이 붓고 점액이 분비된다. 이로 ...


In [38]:
print(df_1.shape)
df_1 = df_1.drop_duplicates(inplace=False)
print(df_1.shape)

(12150274, 11)
(6973723, 11)


In [39]:
# 증상 컬럼이 결측치인 행 찾기
missing_symptom_rows = df_1[df_1['증상'].isnull()]

# 결측치 행의 주상병코드 값 출력
missing_codes = missing_symptom_rows['주상병코드'].unique()

# numpy array를 list로 변환
missing_codes_list = missing_codes.tolist()

# 모든 코드 출력
print("증상 컬럼이 결측치인 행의 주상병코드 값:", missing_codes_list)

증상 컬럼이 결측치인 행의 주상병코드 값: ['T56', 'E11', 'E78', 'K63', 'G31', 'I10', 'B35', 'I11', 'J22', 'D12', 'J40', 'M17', 'S33', 'M79', 'K27', 'M45', 'R41', 'E14', 'B00', 'R07', 'M81', 'R10', 'R52', 'N31', 'B02', 'K92', 'K52', 'K75', 'M85', 'L02', 'E03', 'R42', 'R60', 'R51', 'N40', 'M54', 'R11', 'I67', 'E13', 'K76', 'E55', 'M19', 'R05', 'K73', 'C16', 'I89', 'D44', 'N39', 'L50', 'R82', 'N20', 'I12', 'K12', 'R94', 'T14', 'R35', 'C73', 'D51', 'M80', 'M48', 'I65', 'Z11', 'I25', 'L03', 'L23', 'L20', 'N30', 'M10', 'M25', 'M75', 'M16', 'R53', 'J39', 'I87', 'M13', 'L24', 'R04', 'B34', 'L04', 'T81', 'J34', 'J38', 'G47', 'Z98', 'D23', 'T93', 'K05', 'K60', 'E83', 'G43', 'U09', 'R25', 'Z26', 'H81', 'K37', 'A08', 'K31', 'M15', 'E07', 'N18', 'M02', 'L08', 'Z20', 'J46', 'R50', 'S46', 'R55', 'G58', 'E50', 'B36', 'R32', 'T09', 'R73', 'M46', 'M65', 'I88', 'R30', 'T79', 'Z25', 'E04', 'Z94', 'T78', 'R03', 'K11', 'R20', 'R06', 'L85', 'M06', 'N41', 'M51', 'N28', 'D64', 'K86', 'K62', 'I74', 'H68', 'R00', 'H93', 'M83', 'Z

In [40]:
# 주상병코드가 J, I, H, K, D, E, A로 시작하지 않는 행 제거
df_1 = df_1[df_1['주상병코드'].str.startswith(('J', 'I', 'H', 'K', 'D', 'E', 'A'), na=False)]
print(df_1.shape)

(4195990, 11)


In [41]:
# 증상 컬럼이 결측치인 행 찾기
missing_symptom_rows = df_1[df_1['증상'].isnull()]

# 결측치 행의 주상병코드 값 출력
missing_codes = missing_symptom_rows['주상병코드'].unique()
print("증상 컬럼이 결측치인 행의 주상병코드 값:", missing_codes)

증상 컬럼이 결측치인 행의 주상병코드 값: ['E11' 'E78' 'K63' 'I10' 'I11' 'J22' 'D12' 'J40' 'K27' 'E14' 'K92' 'K52'
 'K75' 'E03' 'I67' 'E13' 'K76' 'E55' 'K73' 'I89' 'D44' 'I12' 'K12' 'D51'
 'I65' 'I25' 'J39' 'I87' 'J34' 'J38' 'D23' 'K05' 'K60' 'E83' 'H81' 'K37'
 'A08' 'K31' 'E07' 'J46' 'E50' 'I88' 'E04' 'K11' 'D64' 'K86' 'K62' 'I74'
 'H68' 'H93' 'E89' 'I68' 'K07' 'E56' 'E86' 'I13' 'D69' 'J35' 'K14' 'I15'
 'E63' 'E58' 'D37' 'K82' 'K83' 'E27' 'D34' 'D17' 'I86' 'I05' 'E43' 'I51'
 'D13' 'K04' 'I77' 'K91' 'D30' 'I78' 'K71' 'D18' 'I69' 'K61' 'D38' 'D72'
 'K13' 'D35' 'I24' 'E87' 'D46' 'E46' 'K77' 'I64' 'I97' 'E02' 'E66' 'D14'
 'D22' 'E23' 'I45' 'I35' 'I82' 'K08' 'E88' 'E79' 'I44' 'E61' 'E53' 'E12'
 'E41' 'D43' 'D21' 'E44' 'E32' 'H40' 'D41' 'H83' 'H13' 'K90' 'D10' 'I34'
 'D53' 'H82' 'H61' 'I99' 'I66' 'I62' 'D15' 'I08' 'I06' 'J69' 'D86' 'D63'
 'D52' 'I27' 'D11' 'I72' 'H92' 'H54' 'I79' 'K02' 'D01' 'E16' 'H26' 'D68'
 'E21' 'H36' 'K09' 'D32' 'D33' 'I61' 'D00' 'K46' 'I23' 'I36' 'I43' 'I22'
 'D84' 'D73' 'E72' 'H90' 'I

In [51]:
print(df_1.shape)
value_counts = df_1['주상병코드'].value_counts()

# value_counts가 10000 이상인 주상병코드만 선택
filtered_codes = value_counts[value_counts >= 10000].index

df_1 = df_1[df_1['주상병코드'].isin(filtered_codes)]
print(df_1.shape)

(4195990, 11)
(3794686, 11)


In [53]:
df_1.to_csv("/content/drive/MyDrive/University/4-2/정보기술학회/data/df_final.csv", index=False, encoding = "utf-8")